**Importações**

In [ ]:
import cv2
import os
import json
import pandas as pd
import numpy as np
import random
from glob import glob
from scipy.io import loadmat
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

**Pré-processamento**

In [ ]:
with open(r'/content/drive/MyDrive/Dataset de roupas/label_descriptions.json') as labels_json:
    labels_df = json.load(labels_json)

print(labels_df['attributes'])

[{'id': 0, 'name': 'above-the-hip (length)', 'supercategory': 'length', 'level': 1}, {'id': 1, 'name': 'hip (length)', 'supercategory': 'length', 'level': 1}, {'id': 2, 'name': 'micro (length)', 'supercategory': 'length', 'level': 1}, {'id': 3, 'name': 'mini (length)', 'supercategory': 'length', 'level': 1}, {'id': 4, 'name': 'above-the-knee (length)', 'supercategory': 'length', 'level': 1}, {'id': 5, 'name': 'knee (length)', 'supercategory': 'length', 'level': 1}, {'id': 6, 'name': 'below the knee (length)', 'supercategory': 'length', 'level': 1}, {'id': 7, 'name': 'midi', 'supercategory': 'length', 'level': 1}, {'id': 8, 'name': 'maxi (length)', 'supercategory': 'length', 'level': 1}, {'id': 9, 'name': 'floor (length)', 'supercategory': 'length', 'level': 1}, {'id': 10, 'name': 'single breasted', 'supercategory': 'opening type', 'level': 1}, {'id': 11, 'name': 'double breasted', 'supercategory': 'opening type', 'level': 1}, {'id': 12, 'name': 'lace up', 'supercategory': 'opening type

In [ ]:
categories=pd.DataFrame(labels_df['categories'])
categories['supercategory'].value_counts()

decorations       10
garment parts      7
upperbody          6
legs and feet      4
wholebody          4
lowerbody          3
others             3
head               3
arms and hands     2
closures           2
neck               1
waist              1
Name: supercategory, dtype: int64

In [ ]:
attributes=pd.DataFrame(labels_df['attributes'])
attributes['supercategory'].value_counts()

silhouette                                     25
textile pattern                                18
textile finishing, manufacturing techniques    17
length                                         10
opening type                                    9
waistline                                       7
animal                                          6
Name: supercategory, dtype: int64

In [ ]:
train_df=pd.read_csv(r'/content/drive/MyDrive/Dataset de roupas/train.csv')
train_df['category']=train_df['ClassId'].str.split('_').str[0]
train_df['attributes']=train_df['ClassId'].str.split('_').str[1:]

In [ ]:
co=[]
for i in range (len(train_df)):
    if len(train_df['attributes'].iloc[i])>0:
        co.append(len(train_df['attributes'].iloc[i]))
    else:
        co.append(0)
train_df['count']=co

In [ ]:
def rle2mask(image_filename,mask_rle,cat,H,W):
   mask=np.ones((H*W),dtype='uint8')*255

   en_list=mask_rle

   cat = np.int32(cat)
   sorted_index = np.argsort(cat)
   
   for i in sorted_index:


     px_list=list(map(int,en_list[i].split(" ")[0::2]))
     
    
     px_count_list=list(map(int,en_list[i].split(" ")[1::2]))
     
    
     cat_list=int(cat[i])
     
     tu=list(map(lambda x, y:(x,y), px_list, px_count_list))
     for p,m in tu:
       
        mask[p:p+m]=cat_list
       
   
   mask = mask.reshape((W,H)).T
   mask=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
   mask=cv2.resize(mask, (512, 512),interpolation=cv2.INTER_NEAREST)
  
   return mask

In [ ]:
image_df = train_df.groupby('ImageId')['EncodedPixels', 'category'].agg(lambda x: list(x))
size_df = train_df.groupby('ImageId')['Height', 'Width'].mean()
image_df = image_df.join(size_df, on='ImageId')

print("Total images: ", len(image_df))
image_df.head()

Total images:  45195


,EncodedPixels,category,Height,Width
ImageId,,,,
00000663ed1ff0c4e0132b9b9ac53f6e.jpg,[6068157 7 6073371 20 6078584 34 6083797 48 60...,"[6, 0, 28, 31, 32, 32, 31, 29, 4]",5214,3676
0000fe7c9191fba733c8a69cfaf962b7.jpg,[2201176 1 2203623 3 2206071 5 2208518 8 22109...,"[33, 1]",2448,2448
0002ec21ddb8477e98b2cbb87ea2e269.jpg,[2673735 2 2676734 8 2679734 13 2682733 19 268...,"[33, 10, 23, 23]",3000,1997
0002f5a0ebc162ecfb73e2c91e3b8f62.jpg,[435 132 1002 132 1569 132 2136 132 2703 132 3...,"[10, 33, 15]",567,400
0004467156e47b0eb6de4aa6479cbd15.jpg,[132663 8 133396 25 134130 41 134868 53 135611...,"[10, 33, 31, 31, 15]",750,500


In [ ]:
image_df2=image_df.reset_index()
image_df2.head()

,ImageId,EncodedPixels,category,Height,Width
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,[6068157 7 6073371 20 6078584 34 6083797 48 60...,"[6, 0, 28, 31, 32, 32, 31, 29, 4]",5214,3676
1,0000fe7c9191fba733c8a69cfaf962b7.jpg,[2201176 1 2203623 3 2206071 5 2208518 8 22109...,"[33, 1]",2448,2448
2,0002ec21ddb8477e98b2cbb87ea2e269.jpg,[2673735 2 2676734 8 2679734 13 2682733 19 268...,"[33, 10, 23, 23]",3000,1997
3,0002f5a0ebc162ecfb73e2c91e3b8f62.jpg,[435 132 1002 132 1569 132 2136 132 2703 132 3...,"[10, 33, 15]",567,400
4,0004467156e47b0eb6de4aa6479cbd15.jpg,[132663 8 133396 25 134130 41 134868 53 135611...,"[10, 33, 31, 31, 15]",750,500


In [ ]:
tr=train_df.groupby('category')['ImageId'].agg(lambda x: list(x)).to_frame('imageid').reset_index()
tr['category']=tr['category'].astype('int')
tr1=tr.sort_values('category', ascending=True)
tr1 = tr1.reset_index(drop=True)
my_dict = {k:v for k,v in zip(tr1['category'], tr1.drop(columns='category').values)}

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(image_df2, test_size = 0.2,random_state=42)
X_train, X_val = train_test_split(X_train, test_size = 0.2,random_state=42)
print('train shape : ',X_train.shape)
print('test shape : ', X_test.shape)
print('val shape : ', X_val.shape)

train shape :  (28924, 5)
test shape :  (9039, 5)
val shape :  (7232, 5)


In [ ]:
def parse_image(i,m):
    image = tf.io.read_file(r'/content/drive/MyDrive/Dataset de roupas/train/'+i)
    image = tf.image.decode_jpeg(image,channels=3)
   
    image = tf.image.resize(image, [256, 256])
    image = image / 255
    
    
    mask = tf.io.read_file(r'/content/drive/MyDrive/Mascara/'+m)
    
    mask = tf.image.decode_png(mask,channels=3)
    
    mask=tf.image.rgb_to_grayscale(mask)
    mask=tf.image.resize(mask,[256,256],method='nearest')
    
    
    mask=tf.where(condition=tf.math.equal(mask, 255), x=tf.cast(46, dtype=mask.dtype), y=mask)
    mask=tf.cast(mask,'float32')
    
    return image,mask

In [ ]:
def random_augmentation(img, mask):
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    

    return img, mask

In [ ]:
def tfdata_train(Tr,batch_size=5):
      f = Tr["ImageId"]
      g = Tr["ImageId"].str.replace('.jpg','.png',regex=False)
      
      train_dataset=tf.data.Dataset.from_tensor_slices((f,g))
      train_dataset= train_dataset.shuffle(len(f))
      train_dataset= train_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
      train_dataset = train_dataset.map(random_augmentation, num_parallel_calls=tf.data.experimental.AUTOTUNE)
      train_dataset = train_dataset.batch(batch_size)
      train_dataset = train_dataset.prefetch(buffer_size=3)  
      return train_dataset

In [ ]:
def tfdata_val(Val,batch_size=5):
    f = Val["ImageId"]
    g = Val["ImageId"].str.replace('.jpg','.png',regex=False)
    val_dataset = tf.data.Dataset.from_tensor_slices((f,g))
    val_dataset = val_dataset.shuffle(len(f))
    val_dataset = val_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size)
    
    return val_dataset

In [ ]:
def tfdata_test(Test,batch_size=5):
  f = Test["ImageId"]
  g = Test["ImageId"].str.replace('.jpg','.png',regex=False)
  test_dataset = tf.data.Dataset.from_tensor_slices((f, g))
  test_dataset = test_dataset.map(parse_image)
  test_dataset = test_dataset.batch(batch_size)
  
  return test_dataset

In [ ]:
train_dataset=tfdata_train(X_train)
val_dataset=tfdata_val(X_val)
test_dataset =tfdata_test(X_test)

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 200
NUM_CLASSES = 47

**Modelo**

In [ ]:
def convolution_block(
    block_input,
    num_filters=16,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

**Treinamento**

In [ ]:
STEPS_PER_EPOCH = X_train.shape[0] // BATCH_SIZE
VALIDATION_STEPS = X_val.shape[0] // BATCH_SIZE

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard('./logs', histogram_freq=1)
callbacks = [

    tensorboard_callback,
    tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('pesos.h5', verbose=1, save_weights_only=True)
     ]

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=["accuracy"],
)

history = model.fit(train_dataset, validation_data=val_dataset, 
                    epochs=6,steps_per_epoch=STEPS_PER_EPOCH,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)

Epoch 1/6
144/144 [==============================] - ETA: 0s - loss: 0.7638 - accuracy: 0.8049
Epoch 00001: saving model to pos_13_epoca.h5
144/144 [==============================] - 2378s 16s/step - loss: 0.7638 - accuracy: 0.8049 - val_loss: 0.7556 - val_accuracy: 0.8028
Epoch 2/6
144/144 [==============================] - ETA: 0s - loss: 0.7210 - accuracy: 0.8107
Epoch 00002: saving model to pos_13_epoca.h5
144/144 [==============================] - 466s 3s/step - loss: 0.7210 - accuracy: 0.8107 - val_loss: 0.8722 - val_accuracy: 0.7684
Epoch 3/6
144/144 [==============================] - ETA: 0s - loss: 0.6846 - accuracy: 0.8176
Epoch 00003: saving model to pos_13_epoca.h5
144/144 [==============================] - 478s 3s/step - loss: 0.6846 - accuracy: 0.8176 - val_loss: 0.8945 - val_accuracy: 0.8001
Epoch 4/6
144/144 [==============================] - ETA: 0s - loss: 0.7516 - accuracy: 0.8050
Epoch 00004: saving model to pos_13_epoca.h5
144/144 [==============================] -

In [ ]:
model.save('deep_lab_final.h5')